In [99]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
#import simplejson as json
import requests
import time
import datetime
import json
from IPython.display import display

import pymysql


In [100]:
URI="dbbikes.cjk4ybuxtkwv.us-east-1.rds.amazonaws.com"
PORT="3306"
DB="dbbikes"
USER="cmcelduff"
PASSWORD="Tullamore1!"

In [101]:
#Connect to database
engine = create_engine("mysql+pymysql://{0}:{1}@{2}:{3}".format(USER, PASSWORD, URI, PORT), echo=True) 
connection = engine.connect()

2023-02-22 13:24:05,787 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-02-22 13:24:05,794 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 13:24:05,897 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-02-22 13:24:05,899 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 13:24:06,096 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-02-22 13:24:06,097 INFO sqlalchemy.engine.Engine [raw sql] {}


In [102]:
sql = """
CREATE DATABASE IF NOT EXISTS dbikes;
"""
engine.execute(sql)

2023-02-22 13:24:07,392 INFO sqlalchemy.engine.Engine 
CREATE DATABASE IF NOT EXISTS dbikes;

2023-02-22 13:24:07,395 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 13:24:07,499 INFO sqlalchemy.engine.Engine COMMIT


In [103]:
# command to check database is running
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

2023-02-22 13:24:07,812 INFO sqlalchemy.engine.Engine SHOW VARIABLES;
2023-02-22 13:24:07,814 INFO sqlalchemy.engine.Engine [raw sql] {}
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('awsauthenticationplugin_max_backoff_delay', '2000')
('awsauthenticationplugin_max_retry_count', '3')
('awsauthenticationplugin_retry_delay', '250')
('back_log', '61')
('basedir', '/rdsdbbin/mysql-8.0.28.R4/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_di

In [104]:
sql_create_schema = "CREATE SCHEMA `dublin_bikes`;"
try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_schema)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-22 13:24:10,473 INFO sqlalchemy.engine.Engine CREATE SCHEMA `dublin_bikes`;
2023-02-22 13:24:10,475 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 13:24:10,570 INFO sqlalchemy.engine.Engine ROLLBACK
(pymysql.err.ProgrammingError) (1007, "Can't create database 'dublin_bikes'; database exists")
[SQL: CREATE SCHEMA `dublin_bikes`;]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [105]:
# command to create table in mysql
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`station` (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contact_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)
"""

try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

    # tom test 
    # tom test 2 

2023-02-22 13:24:11,374 INFO sqlalchemy.engine.Engine 
CREATE DATABASE IF NOT EXISTS dbikes;

2023-02-22 13:24:11,376 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-22 13:24:11,584 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [106]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations/"
DubBike_API = "7f06972a5ed335cf697379627fd13027274927c7"

def write_to_file(text):
    date_underscores = "{:%Y_%m_%d_%H_%M_%S}".format(datetime.datetime.now())
    with open(r"data\bikes_{}".format(date_underscores).replace(" ", "_"), "w") as f:
        f.write(text)

def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get("address"),int(station.get("banking")), int(station.get("bike_stands")), int(station.get("bonus")),station.get("contract_name"), station.get("name"), station.get("number"), station.get("position").get("lat"), station.get("position").get("lng"), station.get("status"))
        engine.execute("INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
    return

def availability_to_db(text):
    stations = json.loads(text)
    #print(availability)
    for station in stations:
        #print(station.get("name"))
        #print(station.get("number"))
        #print(station.get("available_bikes"))
        #print(station.get("available_bike_stands"))
        #print(station.get("last_update"))
        vals = (int(station.get("number")), int(station.get("available_bikes")), int(station.get("available_bike_stands")), int(station.get("last_update")))
        engine.execute("INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)", vals)
    return
        
                                            
def main():
    print(os.path)
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS, params={"apiKey": DubBike_API, "contract" : NAME})
            print(r, now)
            write_to_file(r.text)
            stations_to_db(r.text)
            availability_to_db(r.text)
            time.sleep(5*60)
        except:
            print(traceback.format_exc())
            #if engine is None:
                #pass
        return

if __name__== "__main__":
    main()

<module 'posixpath' from '/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/posixpath.py'>
<Response [200]> 2023-02-22 13:24:11.994675
<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 23, 'available_bikes': 7, 'status': 'OPEN', 'last_update': 1677072154000}
2023-02-22 13:24:12,309 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 13:24:12,310 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield North', 0, 30, 0, 'dublin', 'SMITHFIELD NORTH', 42, 53.349562, -6.278198, 'OPEN')
2023-02-22 13:24:12,552 INFO sqlalchemy.engine.Engine COMMIT
{'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'bank

2023-02-22 13:24:16,739 INFO sqlalchemy.engine.Engine [raw sql] ('Portobello Road', 0, 30, 0, 'dublin', 'PORTOBELLO ROAD', 43, 53.330091, -6.268044, 'OPEN')
2023-02-22 13:24:16,832 INFO sqlalchemy.engine.Engine COMMIT
{'number': 31, 'contract_name': 'dublin', 'name': 'PARNELL STREET', 'address': 'Parnell Street', 'position': {'lat': 53.350929, 'lng': -6.265125}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 13, 'available_bikes': 7, 'status': 'OPEN', 'last_update': 1677072167000}
2023-02-22 13:24:17,022 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 13:24:17,024 INFO sqlalchemy.engine.Engine [raw sql] ('Parnell Street', 0, 20, 0, 'dublin', 'PARNELL STREET', 31, 53.350929, -6.265125, 'OPEN')
2023-02-22 13:24:17,220 INFO sqlalchemy.engine.Engine COMMIT
{'number': 98, 'contract_name': 'dublin', 'name': 'FREDERICK STREET SOUTH', 'address': 'Frederick Street South', 'position': {'lat': 53.34

2023-02-22 13:24:21,219 INFO sqlalchemy.engine.Engine [raw sql] ('Earlsfort Terrace', 0, 30, 0, 'dublin', 'EARLSFORT TERRACE', 11, 53.334295, -6.258503, 'OPEN')
2023-02-22 13:24:21,324 INFO sqlalchemy.engine.Engine COMMIT
{'number': 17, 'contract_name': 'dublin', 'name': 'GOLDEN LANE', 'address': 'Golden Lane', 'position': {'lat': 53.340803, 'lng': -6.267732}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 20, 'available_bikes': 0, 'status': 'OPEN', 'last_update': 1677071888000}
2023-02-22 13:24:21,518 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 13:24:21,519 INFO sqlalchemy.engine.Engine [raw sql] ('Golden Lane', 0, 20, 0, 'dublin', 'GOLDEN LANE', 17, 53.340803, -6.267732, 'OPEN')
2023-02-22 13:24:21,615 INFO sqlalchemy.engine.Engine COMMIT
{'number': 45, 'contract_name': 'dublin', 'name': 'DEVERELL PLACE', 'address': 'Deverell Place', 'position': {'lat': 53.351464, 'lng': -6.255265},

2023-02-22 13:24:25,710 INFO sqlalchemy.engine.Engine [raw sql] ('Lime Street', 0, 40, 0, 'dublin', 'LIME STREET', 62, 53.346026, -6.243576, 'OPEN')
2023-02-22 13:24:25,820 INFO sqlalchemy.engine.Engine COMMIT
{'number': 5, 'contract_name': 'dublin', 'name': 'CHARLEMONT PLACE', 'address': 'Charlemont Street', 'position': {'lat': 53.330662, 'lng': -6.260177}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 28, 'available_bikes': 12, 'status': 'OPEN', 'last_update': 1677072043000}
2023-02-22 13:24:26,035 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 13:24:26,046 INFO sqlalchemy.engine.Engine [raw sql] ('Charlemont Street', 0, 40, 0, 'dublin', 'CHARLEMONT PLACE', 5, 53.330662, -6.260177, 'OPEN')
2023-02-22 13:24:26,272 INFO sqlalchemy.engine.Engine COMMIT
{'number': 97, 'contract_name': 'dublin', 'name': 'KILMAINHAM GAOL', 'address': 'Kilmainham Gaol', 'position': {'lat': 53.342113, 'lng': 

2023-02-22 13:24:30,373 INFO sqlalchemy.engine.Engine [raw sql] ("Sir Patrick's Dun", 0, 40, 0, 'dublin', "SIR PATRICK DUN'S", 58, 53.339218, -6.240642, 'OPEN')
2023-02-22 13:24:30,466 INFO sqlalchemy.engine.Engine COMMIT
{'number': 66, 'contract_name': 'dublin', 'name': 'NEW CENTRAL BANK', 'address': 'New Central Bank', 'position': {'lat': 53.347122, 'lng': -6.234749}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 11, 'available_bikes': 29, 'status': 'OPEN', 'last_update': 1677071882000}
2023-02-22 13:24:30,657 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 13:24:30,659 INFO sqlalchemy.engine.Engine [raw sql] ('New Central Bank', 0, 40, 0, 'dublin', 'NEW CENTRAL BANK', 66, 53.347122, -6.234749, 'OPEN')
2023-02-22 13:24:30,765 INFO sqlalchemy.engine.Engine COMMIT
{'number': 104, 'contract_name': 'dublin', 'name': 'GRANGEGORMAN LOWER (CENTRAL)', 'address': 'Grangegorman Lower (Central)',

2023-02-22 13:24:35,220 INFO sqlalchemy.engine.Engine [raw sql] ('Kevin Street', 0, 40, 0, 'dublin', 'KEVIN STREET', 71, 53.337757, -6.267699, 'OPEN')
2023-02-22 13:24:35,335 INFO sqlalchemy.engine.Engine COMMIT
{'number': 79, 'contract_name': 'dublin', 'name': 'ECCLES STREET EAST', 'address': 'Eccles Street East', 'position': {'lat': 53.358115, 'lng': -6.265601}, 'banking': False, 'bonus': False, 'bike_stands': 27, 'available_bike_stands': 18, 'available_bikes': 9, 'status': 'OPEN', 'last_update': 1677072208000}
2023-02-22 13:24:35,617 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 13:24:35,618 INFO sqlalchemy.engine.Engine [raw sql] ('Eccles Street East', 0, 27, 0, 'dublin', 'ECCLES STREET EAST', 79, 53.358115, -6.265601, 'OPEN')
2023-02-22 13:24:35,712 INFO sqlalchemy.engine.Engine COMMIT
{'number': 69, 'contract_name': 'dublin', 'name': 'GRAND CANAL DOCK', 'address': 'Grand Canal Dock', 'position': {'lat': 53.3426

{'number': 50, 'contract_name': 'dublin', 'name': 'GEORGES LANE', 'address': "George's Lane", 'position': {'lat': 53.35023, 'lng': -6.279696}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 40, 'available_bikes': 0, 'status': 'OPEN', 'last_update': 1677071930000}
2023-02-22 13:24:39,999 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 13:24:40,003 INFO sqlalchemy.engine.Engine [raw sql] ("George's Lane", 0, 40, 0, 'dublin', 'GEORGES LANE', 50, 53.35023, -6.279696, 'OPEN')
2023-02-22 13:24:40,096 INFO sqlalchemy.engine.Engine COMMIT
{'number': 110, 'contract_name': 'dublin', 'name': 'PHIBSBOROUGH ROAD', 'address': 'Phibsborough Road', 'position': {'lat': 53.356307, 'lng': -6.273717}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 24, 'available_bikes': 16, 'status': 'OPEN', 'last_update': 1677072123000}
2023-02-22 13:24:40,390 INFO sqlalchemy.engine.Engine INS

2023-02-22 13:24:44,248 INFO sqlalchemy.engine.Engine [raw sql] ('Parkgate Street', 0, 38, 0, 'dublin', 'PARKGATE STREET', 86, 53.347972, -6.291804, 'OPEN')
2023-02-22 13:24:44,656 INFO sqlalchemy.engine.Engine COMMIT
{'number': 35, 'contract_name': 'dublin', 'name': 'SMITHFIELD', 'address': 'Smithfield', 'position': {'lat': 53.347692, 'lng': -6.278214}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 23, 'available_bikes': 6, 'status': 'OPEN', 'last_update': 1677071706000}
2023-02-22 13:24:44,861 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 13:24:44,864 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield', 0, 30, 0, 'dublin', 'SMITHFIELD', 35, 53.347692, -6.278214, 'OPEN')
2023-02-22 13:24:45,050 INFO sqlalchemy.engine.Engine COMMIT
{'number': 10, 'contract_name': 'dublin', 'name': 'DAME STREET', 'address': 'Dame Street', 'position': {'lat': 53.344007, 'lng': -6.266802}, 'banking': Tr

2023-02-22 13:24:49,336 INFO sqlalchemy.engine.Engine [raw sql] ('St James Hospital (Luas)', 0, 40, 0, 'dublin', 'ST JAMES HOSPITAL (LUAS)', 80, 53.341359, -6.292951, 'OPEN')
2023-02-22 13:24:49,430 INFO sqlalchemy.engine.Engine COMMIT
{'number': 41, 'contract_name': 'dublin', 'name': 'HARCOURT TERRACE', 'address': 'Harcourt Terrace', 'position': {'lat': 53.332763, 'lng': -6.257942}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 20, 'available_bikes': 0, 'status': 'OPEN', 'last_update': 1677071994000}
2023-02-22 13:24:49,650 INFO sqlalchemy.engine.Engine INSERT INTO `dublin_bikes`.`station` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 13:24:49,653 INFO sqlalchemy.engine.Engine [raw sql] ('Harcourt Terrace', 0, 20, 0, 'dublin', 'HARCOURT TERRACE', 41, 53.332763, -6.257942, 'OPEN')
2023-02-22 13:24:49,748 INFO sqlalchemy.engine.Engine COMMIT
{'number': 3, 'contract_name': 'dublin', 'name': 'BOLTON STREET', 'address': 'Bolton Street', 'position': {'lat'

In [107]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations"
r = requests.get(STATIONS, params={"contract" : NAME, "apiKey": DubBike_API})

In [108]:
r.json()

[{'number': 42,
  'contract_name': 'dublin',
  'name': 'SMITHFIELD NORTH',
  'address': 'Smithfield North',
  'position': {'lat': 53.349562, 'lng': -6.278198},
  'banking': False,
  'bonus': False,
  'bike_stands': 30,
  'available_bike_stands': 24,
  'available_bikes': 6,
  'status': 'OPEN',
  'last_update': 1677072264000},
 {'number': 30,
  'contract_name': 'dublin',
  'name': 'PARNELL SQUARE NORTH',
  'address': 'Parnell Square North',
  'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526},
  'banking': False,
  'bonus': False,
  'bike_stands': 20,
  'available_bike_stands': 14,
  'available_bikes': 6,
  'status': 'OPEN',
  'last_update': 1677071941000},
 {'number': 54,
  'contract_name': 'dublin',
  'name': 'CLONMEL STREET',
  'address': 'Clonmel Street',
  'position': {'lat': 53.336021, 'lng': -6.26298},
  'banking': False,
  'bonus': False,
  'bike_stands': 33,
  'available_bike_stands': 30,
  'available_bikes': 3,
  'status': 'OPEN',
  'last_update': 1677071903000},
 

In [109]:
# db_connection = engine.connect(
#     uri="dbbikes.cjk4ybuxtkwv.us-east-1.rds.amazonaws.com",
#     user="cmcelduff",
#     passwd="Tullamore1!",
#     database="dbbikes",
#     port=3306
# )

metadata = sqla.MetaData()
station = sqla.Table("station", metadata,
            sqla.Column('address', sqla.String(256), nullable=False),
            sqla.Column('banking', sqla.Integer),
            sqla.Column('bike_stands', sqla.Integer),
            sqla.Column('bonus', sqla.Integer),
            sqla.Column('contract_name', sqla.String(256)),
            sqla.Column('name', sqla.String(256)),
            sqla.Column('number', sqla.Integer),
            sqla.Column('position_lat', sqla.REAL),
            sqla.Column('position_lng', sqla.REAL),
            sqla.Column('status', sqla.String(256))
)

availability = sqla.Table("availability", metadata,
        sqla.Column('available_bikes', sqla.Integer),
        sqla.Column('available_bikes_stands', sqla.Integer),
        sqla.Column('number', sqla.Integer),
        sqla.Column('last_update', sqla.BigInteger)
)

try:
    station.drop(engine)
    availability.drop(engine)
except:
    pass

metadata.create_all(engine)

2023-02-22 13:24:54,828 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-22 13:24:54,841 INFO sqlalchemy.engine.Engine 
DROP TABLE station
2023-02-22 13:24:54,844 INFO sqlalchemy.engine.Engine [no key 0.00055s] {}
2023-02-22 13:24:54,973 INFO sqlalchemy.engine.Engine ROLLBACK
2023-02-22 13:24:55,159 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-22 13:24:55,170 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-02-22 13:24:55,171 INFO sqlalchemy.engine.Engine [generated in 0.00158s] {'table_schema': 'None', 'table_name': 'station'}
2023-02-22 13:24:55,267 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-02-22 13:24:55,270 INFO sqlalchemy.engine.Engine [cached since 0.1006s ago] {'table_schema': 'None', 'table_name': 'availability'}
2023-02-22 13:24:55,370 INFO sqlalchemy.

OperationalError: (pymysql.err.OperationalError) (1046, 'No database selected')
[SQL: 
CREATE TABLE station (
	address VARCHAR(256) NOT NULL, 
	banking INTEGER, 
	bike_stands INTEGER, 
	bonus INTEGER, 
	contract_name VARCHAR(256), 
	name VARCHAR(256), 
	number INTEGER, 
	position_lat REAL, 
	position_lng REAL, 
	status VARCHAR(256)
)

]
(Background on this error at: https://sqlalche.me/e/14/e3q8)